# Test package

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns
from nn_utilities import keras_utilities, tfp_utilities

sns.set_theme()

Generate fake data for classification.

In [ ]:
n_samples = 2000

x = tf.concat(
    [
        tf.random.normal(shape=(int(n_samples / 2), 1), mean=1., stddev=.5),
        tf.random.normal(shape=(int(n_samples / 2), 1), mean=8., stddev=.5),
    ],
    axis=0
)

y = tf.concat(
    [
        tf.zeros(shape=(int(n_samples / 2))),
        tf.ones(shape=(int(n_samples / 2)))
    ],
    axis=0
)

# Plot data.
fig = plt.figure(figsize=(14, 6))

sns.histplot(
    x=x[:, 0].numpy(),
    stat='density'
)

Create a classification model and train it.

In [ ]:
dropout_rate = 0.3

model = Sequential([
    Dense(units=128, activation='relu'),
    Dropout(rate=dropout_rate),
    Dense(units=64, activation='relu'),
    Dropout(rate=dropout_rate),
    Dense(units=32, activation='relu'),
    Dropout(rate=dropout_rate),
    Dense(units=16),
    Dropout(rate=dropout_rate),
    Dense(units=2, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='sgd'
)

full_history = dict([])

In [ ]:
training_history = model.fit(
    x=x,
    y=y,
    epochs=50,
    verbose=0,
    callbacks=[keras_utilities.NEpochsLogger(10)]
)

full_history = keras_utilities.append_to_full_history(training_history, full_history)

keras_utilities.plot_history(full_history)

Make predictions.

In [ ]:
data_pred = tf.linspace(x.numpy().min(), x.numpy().max(), 500)[..., None]

predicted_prob = model(data_pred)
pred = tf.argmax(predicted_prob, axis=1)

# Plot predictions.
plt.figure(figsize=(14, 6))

sns.scatterplot(
    x=x[:, 0].numpy(),
    y=y.numpy(),
    color=sns.color_palette()[0],
    label='Data'
)

sns.lineplot(
    x=data_pred[:, 0].numpy(),
    y=predicted_prob[:, 1].numpy(),
    color=sns.color_palette()[1],
    label='Predictions'
)

plt.title('Prediction with NN model')

Build the equivalent MC dropout model and make predictions with it.

In [ ]:
mc_dropout_model = keras_utilities.MCDropoutModel(original_model=model).build()

In [ ]:
data_pred = tf.linspace(x.numpy().min(), x.numpy().max(), 500)[..., None]

predicted_prob = mc_dropout_model(data_pred)

# Plot predictions.
plt.figure(figsize=(14, 6))

sns.scatterplot(
    x=x[:, 0].numpy(),
    y=y.numpy(),
    color=sns.color_palette()[0],
    label='Data'
)

sns.lineplot(
    x=data_pred[:, 0].numpy(),
    y=predicted_prob[:, 1].numpy(),
    color=sns.color_palette()[1],
    label='Predictions'
)

plt.title('Prediction with MC dropout model')